# Capturing chessboard images/data for calibration

- Base code has been adapted from: https://www.youtube.com/@outofthebots3122

In [3]:
import numpy as np
import cv2
import time
#creating a window
cv2.namedWindow('Camera Feed')

cam = cv2.VideoCapture(0)

#setting the feed resolution
cam.set(cv2.CAP_PROP_FRAME_WIDTH,720)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT,580)
cam.set(cv2.CAP_PROP_FPS,30)

#initialising time for fps
prev_frame_time = time.time()

# 
cal_image_count = 0 
frame_count = 0 

while True:
    ret, frame = cam.read()
    
    frame_count += 1
    
    #capture an image after every 30 frames and save it
    if frame_count == 30:
        cv2.imwrite("calibration_data/cal_image_"+str(cal_image_count)+".jpg",frame)
        cal_image_count +=1
        frame_count  = 0
        
    #calculating FPS and displaying it
    new_frame_time = time.time()
    fps = 1/(new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(frame,"FPS: " + str(int(fps)), (10,40),cv2.FONT_HERSHEY_PLAIN,3, (100,255,0),2,cv2.LINE_AA)
    cv2.imshow('Camera Feed', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"): 
        break

cam.release()
cv2.destroyAllWindows()

# Generating the calibration and distortion matrix

In [1]:
import numpy as np
import cv2
import glob

#defining the chessboard dimensions
cb_width = 10 #num of horizontal corners
cb_height = 7 #num of vertical corners
cb_square_size = 21


#termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#prepare object points
cb_3D_points = np.zeros((cb_width*cb_height,3),np.float32)
cb_3D_points[:,:2] = np.mgrid[0:cb_width,0:cb_height].T.reshape(-1,2)*cb_square_size

#arrays to store object and image points from all images
list_cb_3D_points=[] #3D points in real world space
list_cb_2D_img_points=[] #2D points in image space

list_images = glob.glob('calibration_data/*.jpg')

for frame_name in list_images:
    img = cv2.imread(frame_name)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #find corners in chessboard with 10x7 corners
    ret, corners = cv2.findChessboardCorners(gray,(10,7), None)

    #if found add object and image points (after refining time)
    if ret == True:
        list_cb_3D_points.append(cb_3D_points)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,1),criteria)
        list_cb_2D_img_points.append(corners2)

        #draw and display corners
        cv2.drawChessboardCorners(img,(cb_width,cb_height),corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(list_cb_3D_points,list_cb_2D_img_points,gray.shape[::-1],None,None)
print("Calibration Matrix: ")
print(mtx)
print("Distortion Matrix: ", dist)

with open('camera_cal.npy','wb') as f:
    np.save(f,mtx)
    np.save(f, dist)

Calibration Matrix: 
[[402.85438258   0.         404.03932491]
 [  0.         401.36977776 243.33030726]
 [  0.           0.           1.        ]]
Distortion Matrix:  [[-0.01118639 -0.11528456 -0.01389512 -0.0023425   0.08385757]]


# Detecting aruco markers and determining the pose

In [2]:
import numpy as np
import cv2
import cv2.aruco as aruco

#marker width and height 
marker_size = 37 #mm

with open("camera_cal.npy",'rb') as f:
    camera_matrix = np.load(f)
    camera_distortion = np.load(f)

#define the aruco marker spec 
#it is 4x4_250 series
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)

cam = cv2.VideoCapture(0)

#defining the camera capture resolution and fps
camera_width = 720
camera_height = 580
camera_frame_rate = 30

cam.set(2,camera_width)
cam.set(4,camera_height)
cam.set(5,camera_frame_rate)

ret, frame = cam.read()

#convert the frame to grayscale
gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

#detect aruco markers
corners, ids , rejected = aruco.detectMarkers(gray_frame,aruco_dict, camera_matrix, camera_distortion)

if ids is not None:
    #draw box around the aruco markers
    aruco.drawDetectedMarkers(frame,corners)

    #get 3D pose of each and every aruco marker
    #get rotational and translational vectors
    rvec, tvec, _objPoints = aruco.estimatePoseSingleMarkers(corners,marker_size,camera_matrix,camera_distortion)

    #draw axis and write ids on all markers
    for marker in range(len(ids)):
        aruco.drawAxis(frame,camera_matrix,camera_distortion,rvec[marker],tvec[marker],10)
        cv2.putText(frame,str(ids[marker][0]),(int(corners[marker][0][0][0])-30,int(corners[marker][0][0][1])),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),2,cv2.LINE_AA)

cv2.imshow('frame',frame)
cv2.waitKey(0)

cam.release()
cv2.destroyAllWindows()

doka


# Detect aruco markers and determine the distance

In [1]:
import numpy as np
import cv2
import cv2.aruco as aruco

#marker width and height 
marker_size = 37 #mm

with open("camera_cal.npy",'rb') as f:
    camera_matrix = np.load(f)
    camera_distortion = np.load(f)

#define the aruco marker spec 
#it is 4x4_250 series
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)


cam = cv2.VideoCapture(0)

#defining the camera capture resolution and fps
camera_width = 720
camera_height = 580
camera_frame_rate = 30

cam.set(2,camera_width)
cam.set(4,camera_height)
cam.set(5,camera_frame_rate)


while True:
    ret, frame = cam.read()

    #convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    #detect aruco markers
    corners, ids , rejected = aruco.detectMarkers(gray_frame,aruco_dict, camera_matrix, camera_distortion)

    if ids is not None:
        #draw box around the aruco markers
        aruco.drawDetectedMarkers(frame,corners)

        #get 3D pose of each and every aruco marker
        #get rotational and translational vectors
        rvec_list_all, tvec_list_all, _objPoints = aruco.estimatePoseSingleMarkers(corners,marker_size,camera_matrix,camera_distortion)
        
        rvec = rvec_list_all[0][0]
        tvec = tvec_list_all[0][0]
        
        aruco.drawAxis(frame,camera_matrix,camera_distortion,rvec,tvec,30)
        tvec_str= "x=%4.0f y=%4.0f z=%4.0f"%(tvec[0],tvec[1],tvec[2])
        cv2.putText(frame,tvec_str,(20,460),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),2,cv2.LINE_AA)

    cv2.imshow('frame',frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()